In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\data_values.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\indicator_type.xlsx',sheet_name='Sheet1')

In [3]:
#defining normalization function for (dam capicity)/(surface rwr)
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(ser1.max()-ser1[i])/(ser1.max()-ser2[i])
        else:
            result[i]=(ser1[i]-ser1.min())/(ser2[i]-ser1.min())
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation function
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result
def addminagg(df,a=0.5):
    size=len(df.columns)
    result=(df.min(axis=1))*a+df.mean(axis=1)*(1-a)
    return result

In [4]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [5]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/20)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr

In [6]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['irwr_percap']=variables.irwr*(10**6)/data.population
sub_indicators['pop_growth']=1+data['pop_growth']/100
sub_indicators['irwr_percap']=sub_indicators['irwr_percap']/sub_indicators['pop_growth']
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
#Dimension 2: access
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['under_stress_pop']=data['under_stress_pop']
#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr
sub_indicators.index=variables['province']

sub_indicators.loc['khuz','damcap_rwr_ratio']=1


In [7]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)    
normalized.drop('pop_growth',axis='columns',inplace=True)

In [8]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(normalized[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=1

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
gw_deficit_bins=pd.qcut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
gw_modifier=pd.Series(index=variables['province'],dtype='float64')
gw_modifier[gw_deficit_bins=='Q1']=0.8
gw_modifier[gw_deficit_bins=='Q2']=0.9
gw_modifier[gw_deficit_bins=='Q3']=1
gw_modifier[gw_deficit_bins=='Q4']=1
normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)


In [9]:
# Aggregating sub_indicators to optain indicators using HYBRID aggregation method on sub_indicators
a=0.5
indicators=pd.DataFrame(index=normalized.index)
indicators['irwr_percap']=normalized['irwr_percap']
indicators['irwr_climate']=  normalized[['rain_coeff_variation','monthly_varicoeff','anomaly_rain','anomaly_temp']] .mean(axis=1)
indicators['water_stress']=  normalized[['withdraw_surf_ratio','withdraw_gw_ratio']] .mean(axis=1)
indicators['deficit_gw']=  normalized[['gw_andef_withdraw','gw_agdef_gwrwr']] .mean(axis=1)
indicators['dependency_gw']=  normalized[['agr_dependency_gw','ruralwat_dependency_gw','urbanwat_dependency_gw','ind_dependency_gw']] .mean(axis=1)
indicators['quality_proxy']=normalized['quality_proxy']
indicators['access_drinking_water']=  normalized[['access_wat_urban','access_wat_rural']] .mean(axis=1)
indicators['under_stress_pop']=normalized['under_stress_pop']
indicators['sanitation']=  normalized[['access_sanitation_urban','access_sanitation_rural','treated_municipal_wastewater']] .mean(axis=1)
indicators['efficiency_agr']=normalized['efficiency_agr']
indicators['modern_irrig']=normalized['modern_irrig']
indicators['efficiency_ind']=normalized['efficiency_ind']
indicators['unaccounted_water']=  normalized[['unaccounted_wat_urban','unaccounted_wat_rural']] .mean(axis=1)
indicators['employment_lost']=normalized['employment_lost_agr']
indicators['dam_capacity']=normalized['damcap_rwr_ratio']

In [32]:
#special wsi aggregating dimensions seperately
indicators_d1=pd.DataFrame(indicators.iloc[:,0:5],index=indicators.index)
indicators_d2=pd.DataFrame(indicators.iloc[:,5:9],index=indicators.index)
indicators_d3=pd.DataFrame(indicators.iloc[:,9:],index=indicators.index)
b=0.5
result_d1=pd.Series(0,index=indicators.index,dtype='float64')
result_d1.loc[indicators_d1['deficit_gw']<0.5]=aggregate(indicators_d1)
result_d1.loc[indicators_d1['deficit_gw']>0.5]=aggregate(indicators_d1.iloc[:,0:4])
indicators_d1['wsd1']=result_d1
indicators_d2['wsd2']=aggregate(indicators_d2,b)
indicators_d3['wsd3']=aggregate(indicators_d3,b)
wsi_special=pd.DataFrame(index=indicators.index)
wsi_special['wsd1']=indicators_d1['wsd1']
wsi_special['wsd2']=indicators_d2['wsd2']
wsi_special['wsd3']=indicators_d3['wsd3']
wsi_special['WSI']=aggregate(wsi_special)
wsi_special.sort_values('WSI',ascending=False)

,wsd1,wsd2,wsd3,WSI
province,,,,
lores,0.849958,0.677628,0.330228,0.597179
kord,0.812367,0.641986,0.366243,0.591384
mazan,0.825178,0.513253,0.453339,0.587078
chba,0.747316,0.658702,0.364510,0.577102
gilan,0.895514,0.571722,0.312728,0.568166
hormo,0.540776,0.495055,0.673569,0.567380
ilam,0.755015,0.673182,0.315716,0.562358
kohbu,0.727568,0.580165,0.338610,0.535814
kersh,0.733084,0.546448,0.342549,0.528243


In [43]:
alt_inds=0
a=0.5
alt_inds=pd.DataFrame(index=normalized.index)
alt_inds['irwr_percap']=normalized['irwr_percap']
alt_inds['irwr_climate']=  normalized[['rain_coeff_variation','monthly_varicoeff','anomaly_rain','anomaly_temp']] .mean(axis=1)
alt_inds['water_stress']=  normalized[['withdraw_surf_ratio','withdraw_gw_ratio']] .mean(axis=1)
alt_inds['deficit_gw']=  normalized[['gw_andef_withdraw','gw_agdef_gwrwr']] .mean(axis=1)
alt_inds['dependency_gw']= normalized[['agr_dependency_gw','ruralwat_dependency_gw','urbanwat_dependency_gw','ind_dependency_gw']] .mean(axis=1)
alt_inds['quality_proxy']=normalized['quality_proxy']
alt_inds['drinking_water']= normalized[['access_wat_urban','access_wat_rural','under_stress_pop']] .mean(axis=1)

alt_inds['sanitation']=normalized[['access_sanitation_urban','access_sanitation_rural','treated_municipal_wastewater']] .mean(axis=1)
alt_inds['agriculture']=normalized[['efficiency_agr','modern_irrig','employment_lost_agr']] .mean(axis=1)

alt_inds['efficiency_ind']=normalized['efficiency_ind']
alt_inds['unaccounted_water']=normalized[['unaccounted_wat_urban','unaccounted_wat_rural']] .mean(axis=1)

alt_inds['dam_capacity']=normalized['damcap_rwr_ratio']

In [44]:
alt_inds_d1=pd.DataFrame(alt_inds.iloc[:,0:5],index=alt_inds.index)
alt_inds_d2=pd.DataFrame(alt_inds.iloc[:,5:8],index=alt_inds.index)
alt_inds_d3=pd.DataFrame(alt_inds.iloc[:,8:],index=alt_inds.index)
b=0.75
res_d1=pd.Series(0,index=alt_inds.index,dtype='float64')
res_d1.loc[alt_inds_d1['deficit_gw']<0.5]=aggregate(alt_inds_d1)
res_d1.loc[alt_inds_d1['deficit_gw']>0.5]=aggregate(alt_inds_d1.iloc[:,0:4])
alt_inds_d1['wsd1']=result_d1
alt_inds_d2['wsd2']=aggregate(alt_inds_d2,b)
alt_inds_d3['wsd3']=aggregate(alt_inds_d3,b)
WSI2=pd.DataFrame(index=alt_inds.index)
WSI2['wsd1']=alt_inds_d1['wsd1']
WSI2['wsd2']=alt_inds_d2['wsd2']
WSI2['wsd3']=alt_inds_d3['wsd3']
WSI2['WSI']=aggregate(WSI2)
WSI2.sort_values('WSI',ascending=False)

,wsd1,wsd2,wsd3,WSI
province,,,,
hormo,0.425871,0.516957,0.675022,0.534484
gilan,0.836615,0.538877,0.264052,0.519221
kord,0.692284,0.668166,0.210035,0.491604
marka,0.330556,0.565633,0.595656,0.489204
azsh,0.370331,0.569767,0.504246,0.477650
lores,0.704845,0.630294,0.189218,0.473089
esfe,0.282303,0.655655,0.511817,0.469561
mazan,0.695563,0.385215,0.359059,0.469080
azgh,0.402277,0.625751,0.393334,0.468206


In [58]:
a=0.5
inds2=pd.DataFrame(index=normalized.index)
inds2['irwr_percap']=normalized['irwr_percap']
inds2['irwr_climate']= normalized[['rain_coeff_variation','monthly_varicoeff','anomaly_rain','anomaly_temp']] .mean(axis=1)
inds2['water_stress']= normalized[['withdraw_surf_ratio','withdraw_gw_ratio']] .mean(axis=1)
inds2['deficit_gw']= normalized[['gw_andef_withdraw','gw_agdef_gwrwr']] .mean(axis=1)
inds2['dependency_gw']= normalized[['agr_dependency_gw','ruralwat_dependency_gw','urbanwat_dependency_gw','ind_dependency_gw']] .mean(axis=1)
inds2['quality_proxy']=normalized['quality_proxy']
inds2['access_drinking_water']= normalized[['access_wat_urban','access_wat_rural']] .mean(axis=1)
inds2['under_stress_pop']=normalized['under_stress_pop']
inds2['sanitation']= normalized[['access_sanitation_urban','access_sanitation_rural','treated_municipal_wastewater']] .mean(axis=1)
inds2['efficiency_agr']=normalized['efficiency_agr']
inds2['modern_irrig']=normalized['modern_irrig']
inds2['efficiency_ind']=normalized['efficiency_ind']
inds2['unaccounted_water']= normalized[['unaccounted_wat_urban','unaccounted_wat_rural']] .mean(axis=1)
inds2['employment_lost']=normalized['employment_lost_agr']
inds2['dam_capacity']=normalized['damcap_rwr_ratio']
inds2_d1=pd.DataFrame(inds2.iloc[:,0:5],index=inds2.index)
inds2_d2=pd.DataFrame(inds2.iloc[:,5:9],index=inds2.index)
inds2_d3=pd.DataFrame(inds2.iloc[:,9:],index=inds2.index)
b=0.25
result_d1=pd.Series(0,index=inds2.index,dtype='float64')
result_d1.loc[inds2_d1['deficit_gw']<0.5]=addminagg(inds2_d1)
result_d1.loc[inds2_d1['deficit_gw']>0.5]=addminagg(inds2_d1.iloc[:,0:4])
inds2_d1['wsd1']=result_d1
inds2_d2['wsd2']=addminagg(inds2_d2,b)
inds2_d3['wsd3']=addminagg(inds2_d3,b)
wsi_spec=pd.DataFrame(index=inds2.index)
wsi_spec['wsd1']=inds2_d1['wsd1']
wsi_spec['wsd2']=inds2_d2['wsd2']
wsi_spec['wsd3']=inds2_d3['wsd3']
wsi_spec['WSI']=addminagg(wsi_special)
wsi_spec.sort_values('WSI',ascending=False)

,wsd1,wsd2,wsd3,WSI
province,,,,
hormo,0.425871,0.462398,0.598945,0.532125
mazan,0.695563,0.468296,0.417276,0.524026
azsh,0.370331,0.514380,0.455892,0.517514
kord,0.692284,0.616216,0.368433,0.484619
chba,0.549182,0.592672,0.345753,0.475709
lores,0.704845,0.607341,0.333808,0.471988
azgh,0.402277,0.574482,0.366179,0.467520
esfe,0.282303,0.631452,0.408207,0.458238
fars,0.469175,0.440457,0.379497,0.456206


In [59]:
simple=pd.DataFrame(index=normalized.index)
simple['irwr_percap']=normalized['irwr_percap']
simple['irwr_climate']=normalized[['rain_coeff_variation','monthly_varicoeff','anomaly_rain','anomaly_temp']].mean(axis=1)
simple['water_stress']= normalized[['withdraw_surf_ratio','withdraw_gw_ratio']] .mean(axis=1)
simple['deficit_gw']= normalized[['gw_andef_withdraw','gw_agdef_gwrwr']] .mean(axis=1)
simple['dependency_gw']= normalized[['agr_dependency_gw','ruralwat_dependency_gw','urbanwat_dependency_gw','ind_dependency_gw']] .mean(axis=1)
simple['quality_proxy']=normalized['quality_proxy']
simple['access_drinking_water']= normalized[['access_wat_urban','access_wat_rural']] .mean(axis=1)
simple['under_stress_pop']=normalized['under_stress_pop']
simple['sanitation']= normalized[['access_sanitation_urban','access_sanitation_rural','treated_municipal_wastewater']] .mean(axis=1)
simple['efficiency_agr']=normalized['efficiency_agr']
simple['modern_irrig']=normalized['modern_irrig']
simple['efficiency_ind']=normalized['efficiency_ind']
simple['unaccounted_water']= normalized[['unaccounted_wat_urban','unaccounted_wat_rural']] .mean(axis=1)
simple['employment_lost']=normalized['employment_lost_agr']
simple['dam_capacity']=normalized['damcap_rwr_ratio']
simple_d1=pd.DataFrame(simple.iloc[:,0:5],index=simple.index)
simple_d2=pd.DataFrame(simple.iloc[:,5:9],index=simple.index)
simple_d3=pd.DataFrame(simple.iloc[:,9:],index=simple.index)
b=0.25
result_d1=pd.Series(0,index=simple.index,dtype='float64')
result_d1.loc[simple_d1['deficit_gw']<0.5]= simple_d1.mean(axis=1)
result_d1.loc[simple_d1['deficit_gw']>0.5]= simple_d1.iloc[:,0:4].mean(axis=1)
simple_d1['wsd1']=result_d1
simple_d2['wsd2']= simple_d2 .mean(axis=1)
simple_d3['wsd3']= simple_d3 .mean(axis=1)
wsi_simple=pd.DataFrame(index=simple.index)
wsi_simple['wsd1']=simple_d1['wsd1']
wsi_simple['wsd2']=simple_d2['wsd2']
wsi_simple['wsd3']=simple_d3['wsd3']
wsi_simple['WSI']=wsi_special.mean(axis=1)
wsi_simple.sort_values('WSI',ascending=False)

,wsd1,wsd2,wsd3,WSI
province,,,,
lores,0.861633,0.702554,0.441745,0.613748
kord,0.821534,0.649389,0.489909,0.602995
mazan,0.836012,0.600725,0.523125,0.594712
gilan,0.898477,0.594285,0.385616,0.587033
chba,0.781320,0.683092,0.456175,0.586908
ilam,0.774525,0.699544,0.408793,0.576568
hormo,0.560759,0.520834,0.692880,0.569195
kohbu,0.782915,0.616564,0.418301,0.545539
kersh,0.750294,0.589253,0.395427,0.537581


In [61]:
WSI_different=pd.DataFrame(index=simple.index)
WSI_different['addgeo']=wsi_special['WSI']
WSI_different['addgeo_alternate']=WSI2['WSI']
WSI_different['addmin']=wsi_spec['WSI']
WSI_different.sort_values('addgeo',ascending=False)

,addgeo,addgeo_alternate,addmin
province,,,
lores,0.597179,0.473089,0.471988
kord,0.591384,0.491604,0.484619
mazan,0.587078,0.469080,0.524026
chba,0.577102,0.459375,0.475709
gilan,0.568166,0.519221,0.449880
hormo,0.567380,0.534484,0.532125
ilam,0.562358,0.446148,0.446142
kohbu,0.535814,0.385279,0.442074
kersh,0.528243,0.461496,0.440065
